In [1]:
import requests
import pandas as pd
import zipfile
import io
from datetime import datetime, timedelta
from sklearn.neighbors import NearestNeighbors
import numpy as np
from PIL import Image
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseUpload

## Retrieving Data

In [2]:
### 1. Bus Stops

def get_bus_stops_data():
    API_KEY = 'MSNUVOUfSEi1b+FVsTYo4A=='
    BASE_URL = "http://datamall2.mytransport.sg/ltaodataservice/BusStops"

    headers = {
        "AccountKey": API_KEY,
        "accept": "application/json"
    }

    skip_value = 0
    all_bus_stops = []

    while True:
        url = f"{BASE_URL}?$skip={skip_value}"
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            data = response.json()
            records = data.get('value', [])
            all_bus_stops.extend(records)

            if len(records) < 500:
                break
            skip_value += 500

        else:
            print(f"Error {response.status_code}: {response.text}")
            break

    all_bus_stops_df = pd.DataFrame(all_bus_stops)
    return all_bus_stops_df

# Call the function and store the DataFrame
bus_stops_df = get_bus_stops_data()

In [27]:
### 2. Bus Monthly Passenger Data (ZIP FILE LINK)

# Define the URL
url = "http://datamall2.mytransport.sg/ltaodataservice/PV/Bus"

# Define the API key (replace 'YOUR_API_KEY' with your actual API key)
api_key = 'MSNUVOUfSEi1b+FVsTYo4A=='

# Set the headers with the API key
headers = {
    "AccountKey": api_key,
    "accept": "application/json"
}

# Make the GET request
response = requests.get(url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    # Get the link to the zip file
    response_json = response.json()
    zip_file_url = response_json['value'][0]['Link']

    # Make a request to download the zip file
    zip_response = requests.get(zip_file_url)
    
    if zip_response.status_code == 200:
        # Use BytesIO for in-memory bytes buffer to handle the zip file
        zip_file_bytes = io.BytesIO(zip_response.content)

        # Use the zipfile library to open the zip file
        with zipfile.ZipFile(zip_file_bytes, 'r') as zip_ref:
            # Extract the file into the current working directory
            zip_ref.extractall("extracted_zip_files")
            
            # Assuming there is only one CSV file in the zip, get its name
            csv_filename = zip_ref.namelist()[0]
            
            # Load the CSV file into a pandas DataFrame
            bus_passengers_df = pd.read_csv(f"extracted_zip_files/{csv_filename}")
else:
    print("Request failed with status code:", response.status_code)

In [4]:
#3. Traffic Incidents

# Define the URL
url = "http://datamall2.mytransport.sg/ltaodataservice/TrafficIncidents"

# Define the API key (replace 'YOUR_API_KEY' with your actual API key)
api_key = 'MSNUVOUfSEi1b+FVsTYo4A=='

# Set the headers with the API key
headers = {
    "AccountKey": api_key,
    "accept": "application/json"
}

# Make the GET request
response = requests.get(url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    # Print the response content
    print(response.json())
else:
    print("Request failed with status code:", response.status_code)
    
traffic_incidents_df = pd.DataFrame(response.json()['value'])

{'odata.metadata': 'http://datamall2.mytransport.sg/ltaodataservice/$metadata#IncidentSet', 'value': [{'Type': 'Vehicle breakdown', 'Latitude': 1.3987804846076621, 'Longitude': 103.89990803573966, 'Message': '(15/4)16:57 Vehicle breakdown on TPE (towards PIE) after Punggol Rd Exit.'}, {'Type': 'Vehicle breakdown', 'Latitude': 1.4353098591094833, 'Longitude': 103.76853588899621, 'Message': '(15/4)16:57 Vehicle breakdown on BKE (towards Woodlands) before Woodlands Rd Exit. Avoid lane 4.'}, {'Type': 'Roadwork', 'Latitude': 1.3350366054480842, 'Longitude': 103.93363331448778, 'Message': '(15/4)16:56 Roadworks on Bedok North Avenue 3 (towards PIE) after PIE. Avoid left lane.'}, {'Type': 'Vehicle breakdown', 'Latitude': 1.3485169516151818, 'Longitude': 103.89844946480385, 'Message': '(15/4)16:55 Vehicle breakdown on KPE (towards TPE) before Airport Rd Entrance.'}, {'Type': 'Vehicle breakdown', 'Latitude': 1.3531280730186832, 'Longitude': 103.78822961253806, 'Message': '(15/4)16:54 Vehicle br

In [5]:
### 4. Speed Bands

def get_speed_band_data():
    # If expired, get new one from LTA datamall
    API_KEY = 'MSNUVOUfSEi1b+FVsTYo4A=='
    BASE_URL = "http://datamall2.mytransport.sg/ltaodataservice/v3/TrafficSpeedBands"

    headers = {
        "AccountKey": API_KEY,
        "accept": "application/json"
    }

    skip_value = 0
    all_records = []

    while True:
        url = f"{BASE_URL}?$skip={skip_value}"
        response = requests.get(url, headers=headers)
        
        if response.status_code == 200:
            data = response.json()
            records = data.get('value', [])
            all_records.extend(records)

            if len(records) < 500:
                break
            skip_value += 500

        else:
            print(f"Error {response.status_code}: {response.text}")
            break
    
    all_records_df = pd.DataFrame(all_records).reset_index(drop = True)
    columns_to_convert = ['StartLat', 'StartLon', 'EndLat', 'EndLon']

    for column in columns_to_convert:
        all_records_df[column] = pd.to_numeric(all_records_df[column], errors='coerce')
    
    return all_records_df

speed_bands_df = get_speed_band_data()

In [6]:
### 5. Taxi Stands

# Define the URL
url = "http://datamall2.mytransport.sg/ltaodataservice/TaxiStands"

# Define the API key (replace 'YOUR_API_KEY' with your actual API key)
api_key = 'MSNUVOUfSEi1b+FVsTYo4A=='

# Set the headers with the API key
headers = {
    "AccountKey": api_key,
    "accept": "application/json"
}

# Make the GET request
response = requests.get(url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    # Print the response content
    print(response.json())
else:
    print("Request failed with status code:", response.status_code)

taxi_stands_df = pd.DataFrame(response.json()['value'])

{'odata.metadata': 'http://datamall2.mytransport.sg/ltaodataservice/$metadata#TaxiStands', 'value': [{'TaxiCode': 'A01', 'Latitude': 1.304294727, 'Longitude': 103.8338467, 'Bfa': 'Yes', 'Ownership': 'LTA', 'Type': 'Stand', 'Name': 'Orchard Rd along driveway of Lucky Plaza'}, {'TaxiCode': 'A05', 'Latitude': 1.304571786, 'Longitude': 103.835547, 'Bfa': 'Yes', 'Ownership': 'Private', 'Type': 'Stand', 'Name': 'Mt Elizabeth Rd at Mt Elizabeth Hospital'}, {'TaxiCode': 'A06', 'Latitude': 1.303519805, 'Longitude': 103.8376729, 'Bfa': 'Yes', 'Ownership': 'LTA', 'Type': 'Stop', 'Name': 'Cairnhill Rd at Cairnhill Nine'}, {'TaxiCode': 'A08', 'Latitude': 1.303709915, 'Longitude': 103.8327402, 'Bfa': 'Yes', 'Ownership': 'Private', 'Type': 'Stand', 'Name': 'Orchard Turn at Wisma Atria Shopping Centre'}, {'TaxiCode': 'A12', 'Latitude': 1.302879021, 'Longitude': 103.8405818, 'Bfa': 'Yes', 'Ownership': 'Private', 'Type': 'Stop', 'Name': 'Cuppage Rd at Starhub Centre'}, {'TaxiCode': 'A13', 'Latitude': 1.

In [7]:
### 6. Approved Road Works (NOT USED)

# Define the URL
url = "http://datamall2.mytransport.sg/ltaodataservice/RoadWorks"

# Define the API key (replace 'YOUR_API_KEY' with your actual API key)
api_key = 'MSNUVOUfSEi1b+FVsTYo4A=='

# Set the headers with the API key
headers = {
    "AccountKey": api_key,
    "accept": "application/json"
}

# Make the GET request
response = requests.get(url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    # Print the response content
    print(response.json())
else:
    print("Request failed with status code:", response.status_code)

road_works_df = pd.DataFrame(response.json()['value'])

{'odata.metadata': 'http://datamall2.mytransport.sg/ltaodataservice/$metadata#RoadWorks', 'value': [{'EventID': 'RMINRM-202311-1078', 'StartDate': '2023-11-23', 'EndDate': '2024-11-20', 'SvcDept': 'SINGTEL - ACCESS NETWORK ENGINEERING DIVISION', 'RoadName': 'ADAM DRIVE', 'Other': 'For details'}, {'EventID': 'RMINRM-202311-1020', 'StartDate': '2023-11-21', 'EndDate': '2024-11-19', 'SvcDept': 'SINGTEL-MANAGED SERVICES MGT and CABLE OPERATIONS', 'RoadName': 'ADAM DRIVE', 'Other': 'For details'}, {'EventID': 'RMINRM-202311-1078', 'StartDate': '2023-11-23', 'EndDate': '2024-11-20', 'SvcDept': 'SINGTEL - ACCESS NETWORK ENGINEERING DIVISION', 'RoadName': 'ADAM PARK', 'Other': 'For details'}, {'EventID': 'RMINRM-202311-1020', 'StartDate': '2023-11-21', 'EndDate': '2024-11-19', 'SvcDept': 'SINGTEL-MANAGED SERVICES MGT and CABLE OPERATIONS', 'RoadName': 'ADAM PARK', 'Other': 'For details'}, {'EventID': 'RMINRM-202402-0105', 'StartDate': '2024-02-01', 'EndDate': '2024-12-31', 'SvcDept': 'SINGTEL-

In [15]:
### 7. Traffic Images

# Define the URL
url = "http://datamall2.mytransport.sg/ltaodataservice/Traffic-Imagesv2"

# Define the API key (replace 'YOUR_API_KEY' with your actual API key)
api_key = 'MSNUVOUfSEi1b+FVsTYo4A=='

# Set the headers with the API key
headers = {
    "AccountKey": api_key,
    "accept": "application/json"
}

# Make the GET request
response = requests.get(url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    # Print the response content
    print(response.json())
else:
    print("Request failed with status code:", response.status_code)

traffic_images_df = pd.DataFrame(response.json()['value'])

{'odata.metadata': 'http://datamall2.mytransport.sg/ltaodataservice/$metadata#Traffic-Imagesv2', 'value': [{'CameraID': '1001', 'Latitude': 1.29531332, 'Longitude': 103.871146, 'ImageLink': 'https://dm-traffic-camera-itsc.s3.ap-southeast-1.amazonaws.com/2024-04-15/17-15/1001_1713_20240415171538_101545.jpg?X-Amz-Security-Token=IQoJb3JpZ2luX2VjEGcaDmFwLXNvdXRoZWFzdC0xIkYwRAIgQ7GDU6EI6pPSyBsJeMy9VMcIlBGA8HMRAVkhgM8dKO8CIE0IwRnz7nHcnz70%2BsTFnf%2F1m4goqWExII3BJwk%2BgTn9KsoFCKD%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FwEQBBoMMzQwNjQ1MzgxMzA0IgxCoVg4YczM9az8%2B48qngXl6oHrtWQtcAeMM8Yg8V9uN1l1CVP%2BrB7Jnx5hiifZmzJBLv0v8won%2FAmaVJrD6o3x%2F7kEQ%2BvAqFPSygefML3s7odFkzfZ07TznNwKp4MWDbXfLqqTzqLIsEpoQIku2WgLP9IkASotvViE9LSnXBstDMPdQg5ahmefwy6yh6Qfrgf24FsBz2osyFdffrc5l3moZy6DD7gkgt%2BUDZGtzu25dI2%2FXellZvrCtMrUDUYuAXYpR6St1N4sFVLiqE5A5rLN3Of584BRuo065bQym2f9IdRBUT9A6TZ1pZXViGm8armP8WceNC3rnr5b%2FVckvT%2FtCw0cIecsrE9i36d3eKqR8c5kUYC3NpHHajQ4TFQ7arPMOiXx7e44x4pE8PH9SUuAIFBylMB5Mp8cM1AbCbax%2Blnhag4zavKx0CWDLN5tCS

In [9]:
### 8. Passenger by Train Station (ZIP FILE LINK)

# Define the URL
url = "http://datamall2.mytransport.sg/ltaodataservice/PV/Train"

# Define the API key (replace 'YOUR_API_KEY' with your actual API key)
api_key = 'MSNUVOUfSEi1b+FVsTYo4A=='

# Set the headers with the API key
headers = {
    "AccountKey": api_key,
    "accept": "application/json"
}

# Make the GET request
response = requests.get(url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    # Get the link to the zip file
    response_json = response.json()
    zip_file_url = response_json['value'][0]['Link']

    # Make a request to download the zip file
    zip_response = requests.get(zip_file_url)
    
    if zip_response.status_code == 200:
        # Use BytesIO for in-memory bytes buffer to handle the zip file
        zip_file_bytes = io.BytesIO(zip_response.content)

        # Use the zipfile library to open the zip file
        with zipfile.ZipFile(zip_file_bytes, 'r') as zip_ref:
            # Extract the file into the current working directory
            zip_ref.extractall("extracted_zip_files")
            
            # Assuming there is only one CSV file in the zip, get its name
            csv_filename = zip_ref.namelist()[0]
            
            # Load the CSV file into a pandas DataFrame
            train_passengers_df = pd.read_csv(f"extracted_zip_files/{csv_filename}")
else:
    print("Request failed with status code:", response.status_code)

In [10]:
### 9. Bus Arrival

# Define the URL
url = "http://datamall2.mytransport.sg/ltaodataservice/BusArrivalv2"

# Define the API key (replace 'YOUR_API_KEY' with your actual API key)
api_key = 'MSNUVOUfSEi1b+FVsTYo4A=='

bus_arrivals_list = []

# List of BusStopCodes to query
busStopCodes = bus_stops_df['BusStopCode']

# Loop through each BusStopCode and make a request
for code in busStopCodes:
    # Set the headers with the API key
    headers = {
        "AccountKey": api_key,
        "accept": "application/json"
    }

    # Set the parameters with the BusStopCode
    params = {
        "BusStopCode": code
    }

    # Make the GET request
    response = requests.get(url, headers=headers, params=params)

    # Check if the request was successful
    if response.status_code == 200:
        # Print the response content
       bus_arrivals_list.append(response.json())
    else:
        print("Request failed with status code:", response.status_code, "for BusStopCode:", code)

In [11]:
train_station_codes = pd.read_excel('static_data/Train Station Codes and Chinese Names.xls', engine='xlrd')

## Data Transformation
#### Refer to ERD for dataframe columns

In [12]:
### Road Entity

road = speed_bands_df[['LinkID', 'RoadName', 'RoadCategory', 'StartLon', 'StartLat', 'EndLon', 'EndLat']]

### Speed Band Entity

speed_band = speed_bands_df[['LinkID', 'SpeedBand']]
# Add a new column with the current time
current_time = datetime.now()
speed_band['Time'] = current_time

/var/folders/tl/_qf64lp53pxdk06l84lqd6th0000gn/T/ipykernel_16754/592780591.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  speed_band['Time'] = current_time


In [16]:
### Traffic Image Entity

traffic_image = traffic_images_df.copy()

## MAP LINKID TO TRAFFIC IMAGE

# First, prepare the data for NearestNeighbors
coords_traffic_image = traffic_image[['Latitude', 'Longitude']]

# Take each LinkID as middle of the Link segment
coords_speed_bands = speed_bands_df[['StartLat', 'StartLon', 'EndLat', 'EndLon']]
coords_speed_bands['Latitude'] = (speed_bands_df['StartLat'] + speed_bands_df['EndLat']) / 2
coords_speed_bands['Longitude'] = (speed_bands_df['StartLon'] + speed_bands_df['EndLon']) / 2
coords_speed_bands = coords_speed_bands[['Latitude', 'Longitude']]

# Fit the NearestNeighbors model on the speed bands coordinates
nn = NearestNeighbors(n_neighbors=1).fit(coords_speed_bands)

# Find the closest LinkID for each traffic image
distances, indices = nn.kneighbors(coords_traffic_image)

# Assign the closest LinkID to each row in traffic_image
traffic_image['LinkID'] = speed_bands_df.iloc[indices.flatten()]['LinkID'].values

traffic_image['Time'] = datetime.now()

# Load service account credentials
credentials = service_account.Credentials.from_service_account_file('client_secrets.json')
drive_service = build('drive', 'v3', credentials=credentials)

def upload_image_to_google_drive(image, folder_id, filename):
    # Create file metadata
    file_metadata = {
        'name': filename,
        'parents': [folder_id]
    }

    # Convert PIL image to bytes
    img_byte_arr = io.BytesIO()
    image.save(img_byte_arr, format='JPEG')
    media_body = MediaIoBaseUpload(img_byte_arr, mimetype='image/jpeg')

    # Upload file to Google Drive
    uploaded_file = drive_service.files().create(body=file_metadata, media_body=media_body).execute()

folder_id = "1EcGRHhHU1ZEHf_xtJ2fsP9FIFSSk5u7c"

# Iterate through traffic images
for index, row in traffic_image.iterrows():
    image_url = row['ImageLink']
    response = requests.get(image_url)
    image = Image.open(io.BytesIO(response.content))

    filename = f'Traffic_Image_{row["CameraID"]}_{row["Time"]}.jpg'
    upload_image_to_google_drive(image, folder_id, filename)


/var/folders/tl/_qf64lp53pxdk06l84lqd6th0000gn/T/ipykernel_16754/1823769319.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  coords_speed_bands['Latitude'] = (speed_bands_df['StartLat'] + speed_bands_df['EndLat']) / 2
/var/folders/tl/_qf64lp53pxdk06l84lqd6th0000gn/T/ipykernel_16754/1823769319.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  coords_speed_bands['Longitude'] = (speed_bands_df['StartLon'] + speed_bands_df['EndLon']) / 2


In [18]:
# Save GoogleDrive image links to traffic_image.csv

def list_images_in_folder(folder_id):
    image_links = []
    page_token = None

    while True:
        # Query files in the folder
        response = drive_service.files().list(q=f"'{folder_id}' in parents and mimeType='image/jpeg'",
                                              fields='nextPageToken, files(id, webContentLink)',
                                              pageToken=page_token).execute()
        # Extract image links from the response
        for file in response.get('files', []):
            image_links.append(file.get('webContentLink'))

        # Check if there are more files to retrieve
        page_token = response.get('nextPageToken', None)
        if page_token is None:
            break

    return image_links

folder_id = "1EcGRHhHU1ZEHf_xtJ2fsP9FIFSSk5u7c"
image_links = list_images_in_folder(folder_id)

traffic_image['ImageLink'] = image_links


In [19]:
### Bus Stop Entity

bus_stop = bus_stops_df.copy()

## MAP LINKID TO BUS STOP

# First, prepare the data for NearestNeighbors
coords_bus_stops = bus_stop[['Latitude', 'Longitude']]

# Take each LinkID as middle of the Link segment
coords_speed_bands = speed_bands_df[['StartLat', 'StartLon', 'EndLat', 'EndLon']]
coords_speed_bands['Latitude'] = (speed_bands_df['StartLat'] + speed_bands_df['EndLat']) / 2
coords_speed_bands['Longitude'] = (speed_bands_df['StartLon'] + speed_bands_df['EndLon']) / 2
coords_speed_bands = coords_speed_bands[['Latitude', 'Longitude']]

# Fit the NearestNeighbors model on the speed bands coordinates
nn = NearestNeighbors(n_neighbors=1).fit(coords_speed_bands)

# Find the closest LinkID for each bus stop
distances, indices = nn.kneighbors(coords_bus_stops)

# Assign the closest LinkID to each row in bus_stop_df
bus_stop['LinkID'] = speed_bands_df.iloc[indices.flatten()]['LinkID'].values

/var/folders/tl/_qf64lp53pxdk06l84lqd6th0000gn/T/ipykernel_16754/662224760.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  coords_speed_bands['Latitude'] = (speed_bands_df['StartLat'] + speed_bands_df['EndLat']) / 2
/var/folders/tl/_qf64lp53pxdk06l84lqd6th0000gn/T/ipykernel_16754/662224760.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  coords_speed_bands['Longitude'] = (speed_bands_df['StartLon'] + speed_bands_df['EndLon']) / 2


In [20]:
### Taxi Stand Entity

taxi_stand = taxi_stands_df.copy()

## MAP LINKID TO TAXI STAND

# First, prepare the data for NearestNeighbors
coords_taxi_stands = taxi_stand[['Latitude', 'Longitude']]

# Take each LinkID as middle of the Link segment
coords_speed_bands = speed_bands_df[['StartLat', 'StartLon', 'EndLat', 'EndLon']]
coords_speed_bands['Latitude'] = (speed_bands_df['StartLat'] + speed_bands_df['EndLat']) / 2
coords_speed_bands['Longitude'] = (speed_bands_df['StartLon'] + speed_bands_df['EndLon']) / 2
coords_speed_bands = coords_speed_bands[['Latitude', 'Longitude']]

# Fit the NearestNeighbors model on the speed bands coordinates
nn = NearestNeighbors(n_neighbors=1).fit(coords_speed_bands)

# Find the closest LinkID for each taxi stand
distances, indices = nn.kneighbors(coords_taxi_stands)

# Assign the closest LinkID to each row in taxi_stand
taxi_stand['LinkID'] = speed_bands_df.iloc[indices.flatten()]['LinkID'].values

/var/folders/tl/_qf64lp53pxdk06l84lqd6th0000gn/T/ipykernel_16754/1002697068.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  coords_speed_bands['Latitude'] = (speed_bands_df['StartLat'] + speed_bands_df['EndLat']) / 2
/var/folders/tl/_qf64lp53pxdk06l84lqd6th0000gn/T/ipykernel_16754/1002697068.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  coords_speed_bands['Longitude'] = (speed_bands_df['StartLon'] + speed_bands_df['EndLon']) / 2


In [21]:
### Traffic Incident Entity

traffic_incident = traffic_incidents_df.copy()

## MAP LINKID TO TAXI STAND

# First, prepare the data for NearestNeighbors
coords_traffic_incidents = traffic_incident[['Latitude', 'Longitude']]

# Take each LinkID as middle of the Link segment
coords_speed_bands = speed_bands_df[['StartLat', 'StartLon', 'EndLat', 'EndLon']]
coords_speed_bands['Latitude'] = (speed_bands_df['StartLat'] + speed_bands_df['EndLat']) / 2
coords_speed_bands['Longitude'] = (speed_bands_df['StartLon'] + speed_bands_df['EndLon']) / 2
coords_speed_bands = coords_speed_bands[['Latitude', 'Longitude']]

# Fit the NearestNeighbors model on the speed bands coordinates
nn = NearestNeighbors(n_neighbors=1).fit(coords_speed_bands)

# Find the closest LinkID for each taxi stand
distances, indices = nn.kneighbors(coords_traffic_incidents)

# Assign the closest LinkID to each row in traffic_incident
traffic_incident['LinkID'] = speed_bands_df.iloc[indices.flatten()]['LinkID'].values
traffic_incident['Time'] = datetime.now()

/var/folders/tl/_qf64lp53pxdk06l84lqd6th0000gn/T/ipykernel_16754/2211210194.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  coords_speed_bands['Latitude'] = (speed_bands_df['StartLat'] + speed_bands_df['EndLat']) / 2
/var/folders/tl/_qf64lp53pxdk06l84lqd6th0000gn/T/ipykernel_16754/2211210194.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  coords_speed_bands['Longitude'] = (speed_bands_df['StartLon'] + speed_bands_df['EndLon']) / 2


In [22]:
### Bus Density Entity --> arriving within 5 min counted as +1 to the density

# List to store the processed data
data = []

# Get the current time
current_time = datetime.now()

# Iterate through the list of bus arrivals
for bus_stop_info in bus_arrivals_list:
    bus_stop_code = bus_stop_info['BusStopCode']
    num_buses = 0  # Initialize the number of buses arriving within the next minute

    # Iterate through the services at the bus stop
    for service in bus_stop_info['Services']:
        for key in ['NextBus', 'NextBus2', 'NextBus3']:
            next_bus = service.get(key, {})
            arrival_time_str = next_bus.get('EstimatedArrival', '')
            
            # Check if there's an arrival time
            if arrival_time_str:
                # Convert the arrival time to a datetime object
                arrival_time = datetime.strptime(arrival_time_str, '%Y-%m-%dT%H:%M:%S+08:00')
                
                # Check if the bus is arriving within the next minute
                if current_time <= arrival_time <= current_time + timedelta(minutes=5):
                    num_buses += 1

    # Add the data for this bus stop to the list
    data.append({'BusStopCode': bus_stop_code, 'NumBuses': num_buses, 'Time': current_time})

# Convert the list of data to a DataFrame
bus_density = pd.DataFrame(data)

In [43]:
### Bus Passenger Density Entity

bus_passenger_density = bus_passengers_df[['YEAR_MONTH', 'DAY_TYPE', 'PT_CODE',
       'TOTAL_TAP_IN_VOLUME', 'TOTAL_TAP_OUT_VOLUME']]

bus_passenger_density.columns =  ['Year_Month', 'Day_Type', 'BusStopCode',
       'TOTAL_TAP_IN_VOLUME', 'TOTAL_TAP_OUT_VOLUME']

# Pivoting the DataFrame to reshape it based on 'Day_Type'
bus_passenger_density = bus_passenger_density.pivot_table(index=['Year_Month', 'BusStopCode'], 
                                   columns='Day_Type', 
                                   values=['TOTAL_TAP_IN_VOLUME', 'TOTAL_TAP_OUT_VOLUME'],
                                   aggfunc='sum').reset_index()

# Rename the columns
bus_passenger_density.columns = ['YearMonth', 'BusStopCode', 'WeekdayTapIn', 'WeekendTapIn', 
                    'WeekdayTapOut', 'WeekendTapOut']

valid_bus_stops = [int(stop) for stop in bus_stop['BusStopCode']]

bus_passenger_density = bus_passenger_density[bus_passenger_density['BusStopCode'].isin(valid_bus_stops)]


In [24]:
### MRT Train Station Entity

mrt_train_station =  train_station_codes[['stn_code', 'mrt_station_english',
       'mrt_line_english']]

mrt_train_station.columns = ['TrainStationCode', 'Name', 'MRTLine']

In [25]:
### Train Passenger Density Entity

mrt_train_passenger_density = train_passengers_df[['YEAR_MONTH', 'DAY_TYPE', 'PT_CODE',
       'TOTAL_TAP_IN_VOLUME', 'TOTAL_TAP_OUT_VOLUME']]

mrt_train_passenger_density.columns =  ['Year_Month', 'Day_Type', 'TrainStationCode',
       'TOTAL_TAP_IN_VOLUME', 'TOTAL_TAP_OUT_VOLUME']

# Pivoting the DataFrame to reshape it based on 'Day_Type'
mrt_train_passenger_density = mrt_train_passenger_density.pivot_table(index=['Year_Month', 'TrainStationCode'], 
                                   columns='Day_Type', 
                                   values=['TOTAL_TAP_IN_VOLUME', 'TOTAL_TAP_OUT_VOLUME'],
                                   aggfunc='sum').reset_index()

# Rename the columns
mrt_train_passenger_density.columns = ['YearMonth', 'TrainStationCode', 'WeekdayTapIn', 'WeekendTapIn', 
                    'WeekdayTapOut', 'WeekendTapOut']

valid_train_stations = mrt_train_station['TrainStationCode']

mrt_train_passenger_density = mrt_train_passenger_density[mrt_train_passenger_density['TrainStationCode'].isin(valid_train_stations)]


## Save transformed data
#### Save into snapshot_data folder

In [26]:
# Define the folder path
folder_path = 'snapshot_data/'

# Use pandas to save each DataFrame to a CSV file in the 'mock_data' folder
road.to_csv(folder_path + 'road.csv', index=False)
speed_band.to_csv(folder_path + 'speed_band.csv', index=False)
traffic_image.to_csv(folder_path + 'traffic_image.csv', index=False)
bus_stop.to_csv(folder_path + 'bus_stop.csv', index=False)
bus_density.to_csv(folder_path + 'bus_density.csv', index=False)
bus_passenger_density.to_csv(folder_path + 'bus_passenger_density.csv', index=False)
taxi_stand.to_csv(folder_path + 'taxi_stand.csv', index=False)
mrt_train_station.to_csv(folder_path + 'mrt_train_station.csv', index=False)
mrt_train_passenger_density.to_csv(folder_path + 'mrt_train_passenger_density.csv', index=False)
traffic_incident.to_csv(folder_path + 'traffic_incident.csv', index=False)